In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import time

import json

from skrub import TableReport
from ollama import Client
from pdfquery import PDFQuery
from pprint import pprint

import bid_utils

In [2]:
## load the dfs built in 1_collect_files
rep_drive_ATAE =  r"C:\Users\jch_m\ATAE"
rep_data =        r"C:\DocPerso\Dev_Python\Data\ATAE"
rep_data_output = r"C:\DocPerso\Dev_Python\ProjetsPerso\BidManagement\data_output"

path_df_EBP = os.path.join(rep_data_output, "df_EBP.pkl")
df_EBP = pd.read_pickle(path_df_EBP)

path_df_consult = os.path.join(rep_data_output, "df_consult.pkl")
df_consult = pd.read_pickle(path_df_consult)

path_df_rejet = os.path.join(rep_data_output, "df_rejet.pkl")
df_rejet = pd.read_pickle(path_df_rejet)


In [3]:
# Merge df_consult and df_EBP pour sync EBP ID on the mission/files
# note: no need to sync EBP with rejet, because none of the Rejet file match an EBP entry (dans la liste des repertoires et fichiers). normal ?

# Define the columns to match on
match_columns = ['SPS Name', 'Ville', 'Entreprise', 'Mission']

df_consult_ebp = pd.merge(df_consult, df_EBP, on=match_columns, how='left', suffixes=('_consult', '_ebp'))
df_consult_ebp['ID EBP'] = df_consult_ebp['ID EBP_ebp'].fillna("no EBP")
df_consult_ebp = df_consult_ebp.drop(columns=['ID EBP_ebp','ID EBP_consult', 'statut_ebp'])

#df_consult_ebp['file_name'] = df_consult_ebp['file_path'].str.split(r'\\').str[-1].strip()
df_consult_ebp['file_name'] = df_consult_ebp['file_path'].str.split(r'\\').str[-1].str.strip()

In [4]:
# define a unique 'no EBP xx'  for each combinaison of SPS+Ville+Entreprise+Mission, for all related files
#
mask = df_consult_ebp["ID EBP"] == "no EBP"

# Create the combined series only for the masked rows
combined_series_for_update = (
    df_consult_ebp.loc[mask, 'SPS Name'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Ville'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Entreprise'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Mission'].astype(str)
)

# Generate unique IDs for this series
unique_ids, _ = pd.factorize(combined_series_for_update)

# Assign back to the original DataFrame using the mask
df_consult_ebp.loc[mask, "ID EBP"] = "no EBP " + (unique_ids + 1).astype(str)


In [5]:
# Add columns "AO_docs" (True/False), "AO_Doc_type"(CCTP, CCAP, RC, AAPC, Memo_tech), "Commande"(True/False)

df_consult_ebp['AO_docs'] = (
    (df_consult_ebp['file_path'].str.lower().str.contains('devis') & df_consult_ebp['file_path'].str.lower().str.contains(r'\\consul')) | 
    (df_consult_ebp['file_path'].str.lower().str.contains(r'\\adm') & df_consult_ebp['file_path'].str.lower().str.contains(r'\\consul'))
)

# Set AO_doc_type
# Initialize all with default value
df_consult_ebp['AO_doc_type'] = 'no type'
mask = df_consult_ebp['AO_docs'] == True
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('CCTP', case=False, na=False), 'AO_doc_type'] = 'CCTP'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('CCP', case=False, na=False), 'AO_doc_type'] = 'CCTP'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('DCE', case=False, na=False), 'AO_doc_type'] = 'CCTP'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('Lettre', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('courier', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('consult', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('CCAP', case=False, na=False), 'AO_doc_type'] = 'CCAP'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('AAPC', case=False, na=False), 'AO_doc_type'] = 'AAPC'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('PA-', case=False, na=False), 'AO_doc_type'] = 'Procedure Adaptee'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('RC', case=False, na=False), 'AO_doc_type'] = 'RC'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('glement', case=False, na=False), 'AO_doc_type'] = 'RC'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('planning', case=False, na=False), 'AO_doc_type'] = 'Planning'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('phasag', case=False, na=False), 'AO_doc_type'] = 'Planning'

In [6]:
list_file_AO_notype = ""
mask2 = (df_consult_ebp['AO_docs'] == True) &  (df_consult_ebp['AO_doc_type'] == 'no type')
list_file_AO_notype = df_consult_ebp.loc[mask2, 'file_name']
for file in list_file_AO_notype:
    if "plan " in file.lower() or "plans" in file.lower() or "coupe" in file.lower() or "vue" in file.lower() or "facade" in file.lower() or "lot" in file.lower():
        continue
    print(file)

Estimation ESQ-DIAG Modulaire Pierre et Marie Curie.pdf
CSPS Contexte_Filières_HLX.pdf
Heinlex Filieres-Programme-rev2.pdf
22062-PA8b-2023-06-22.pdf
2023-12-12-A1_existant.pdf
2023-12-12-A1_projet.pdf
M22.054-CR01-20230907.pdf
M22054-21-voirie.pdf
Fiche identité CSPS.pdf
Attestation 2021- 52-301 - SARL ATAE.pdf
2-Note méthodologique VENDEE EAU.pdf
3 DE SPS_230321 6 sites.pdf
3 DE SPS_230321 RECAPITULATIF.pdf
4-Attestation sur l'Honneur.pdf
5-Attestation Assurance 2021 ATAE.pdf
6-DC1.pdf
7-DC2 et annexes.pdf
Programme technique.pdf
ANNEXE tableau financier TVX base.pdf
ANNEXE tableau financier TVX options.pdf
AVP TOITURE CHLVO.pdf
03b DTA _  _ Travaux toiture Lege.pdf
04 Mémoire technique.pdf
Mémoire technique Blandin Travaux toiture Lege.pdf
Cahier charges mission CSPS_Boucardière - ATAE.pdf
Cahier charges mission CSPS_Boucardière.pdf
MACHECOUL - MASSE - 2017 02 28.pdf
MACHECOUL - MASSE - 2017 02 28.pdf
00-Courriersolicitation.pdf
02 Ficherenseignements SPS.pdf
04-BPU_Petit-Gast&Beauso

In [7]:
#df_consult_ebp[df_consult_ebp['file_name'] == "PA-SPSLORIENT.pdf"]
df_consult_ebp[df_consult_ebp['file_name'].str.strip() == "PA-SPSLORIENT.pdf"]

,SPS Name,Ville,Entreprise,Mission,statut_consult,file_path,ID EBP,file_name,AO_docs,AO_doc_type
106919,Nicolas POTIER - Nicolas POTIER,LORIENT,MH,Construction immeuble 20 logements,Devis,C:\Users\jch_m\ATAE\Nicolas POTIER - Nicolas P...,no EBP 17307,PA-SPSLORIENT.pdf,True,Procedure Adaptee


In [8]:
df_consult_ebp['AO_docs'].value_counts()

AO_docs
False    152162
True       3095
Name: count, dtype: int64

In [9]:
df_consult_ebp['AO_doc_type'].value_counts()

AO_doc_type
no type              153673
CCTP                    709
RC                      457
CCAP                    200
Planning                 92
Lettre Consult           85
AAPC                     30
Procedure Adaptee        11
Name: count, dtype: int64

In [10]:
df_consult_ebp.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155257 entries, 0 to 155256
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   SPS Name        155257 non-null  object
 1   Ville           155257 non-null  object
 2   Entreprise      155257 non-null  object
 3   Mission         155257 non-null  object
 4   statut_consult  155257 non-null  object
 5   file_path       155257 non-null  object
 6   ID EBP          155257 non-null  object
 7   file_name       155257 non-null  object
 8   AO_docs         155257 non-null  bool  
 9   AO_doc_type     155257 non-null  object
dtypes: bool(1), object(9)
memory usage: 10.8+ MB


In [11]:
TableReport(df_consult_ebp)

Processing column  10 / 10


,,,,,,,,,,


In [12]:
# Creation d'un DF_consult_elevated avec une ligne par "EBP ID", 
# et l'ajout des features resultants de l'extraction 

df_consult_ao = df_consult_ebp[df_consult_ebp['AO_docs']==True]
df_consult_elevated = df_consult_ao.drop_duplicates(subset=["ID EBP"], keep="first")

# Suppression des colonnes desormais inutiles
df_consult_elevated = df_consult_elevated.drop(columns=['file_path','file_name','AO_docs','AO_doc_type'])

df_consult_elevated['lieu'] = ""
df_consult_elevated['type travaux'] = ""
df_consult_elevated['planning'] = ""
df_consult_elevated['duree travaux'] = 0
df_consult_elevated['prix travaux'] = 0.0
df_consult_elevated['maitre ouvrage'] = ""
df_consult_elevated['maitre oeuvre'] = ""

In [13]:
df_consult_elevated.info()


<class 'pandas.core.frame.DataFrame'>
Index: 923 entries, 0 to 155112
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SPS Name        923 non-null    object 
 1   Ville           923 non-null    object 
 2   Entreprise      923 non-null    object 
 3   Mission         923 non-null    object 
 4   statut_consult  923 non-null    object 
 5   ID EBP          923 non-null    object 
 6   lieu            923 non-null    object 
 7   type travaux    923 non-null    object 
 8   planning        923 non-null    object 
 9   duree travaux   923 non-null    int64  
 10  prix travaux    923 non-null    float64
 11  maitre ouvrage  923 non-null    object 
 12  maitre oeuvre   923 non-null    object 
dtypes: float64(1), int64(1), object(11)
memory usage: 101.0+ KB


# Test de modeles pour resumer et extraire l'info un CCTP 

In [23]:
prompt_role = "tu es un assistant pour analyser les appels d'offres, extraire les informations demandees en suivant les instructions"

prompt_task_resumer = "En Francais, extrait du texte les elements suivants : \n\
   'Nom Chantier': scope du projet, objet du chantier ;\n\
   'Lieu du Chantier': ville, Commune, Departement, Rue ; \n\
   'Maitre ouvrage': nom du Maitre d'ouvrage du projet ;\n\
   'Maitre oeuvre': nom du maitre d'oeuvre du chantier  ;\n\
   'Type de travaux': type et nature du travaux du chantier exemple: amenagement, construction ;\n \
   'Planning previsionnel': dates du chantier par lot, par phase ;\n\
   'Prix des travaux (en euros)': exemple <10_000> ;\n\
   'Duree Previsionnelle des Travaux (en mois)': exemple <8> ;\n\
   'Categorie operation SPS':  I, II, ou III ;\n\
Réponds en respectant la structure ci-dessus \n\
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. \n\
Donne des reponses chiffrées et quantifiées lorsque c'est possible. \n\
Ne mentionne pas **Autres informations:**  \n\
La reponse doit avoir un maximum 900 mots \n \
Texte: "

prompt_task_json = "En Francais, Formate les informations extraites et dans le texte, avec une structure JSON, en respectant strictement les clefs et types specifies tels que \n \
{ \
  \"Nom Chantier\": \"<string>\", \
  \"Lieu du Chantier\": \"<string>\",\
  \"Maitre ouvrage\": \"<string>\", \
  \"Maitre oeuvre\": \"<string>\", \
  \"Type de Travaux\": \"<string>\", \
  \"Planning previsionnel\": \"<string>\", \
  \"Prix des travaux (en euros)\": \"<integer>\" , \
  \"Durée Prévisionnelle des Travaux (en mois)\": \"<integer>\" ,\
} \n\
Donne des reponses uniquement pour ces clefs. en chiffre lorsque c'est possible. Si pas d'info, laisser vide  \n "



In [15]:
# Init Ollama, with list of model, prompt

ollama_url = "http://localhost:11434"

# Get an ollama client
llmclient = Client(host=ollama_url)

model_options = {
    "num_predict": 1300,  # max number of tokens to predict
    "temperature": 0.1,
    "top_p": 0.9,
}


# 'mistral-small3.1=14G  llama3.2:latest=2G gemma3:4b=3.3G
#list_model = ["gemma3:4b", "llama3.2", "minicpm-v", "mistral-small3.1"]
list_model = ["gemma3:12b", "gemma2:9b", "llama3.1:8b", "mistral:7b"]



In [25]:
## Run all models for the same task (prompt + file) for comparison
##

def run_list_model():

    # Loop on the list of models specified
    for i in range(len(list_model)):
        model_name = list_model[i]
        print("\n==========================================")
        print(f"===== Test model :{i+1}/{len(list_model)}: {model_name} =========\n")
        
        #Resumer
        prompt_full = prompt_task_resumer + "".join(text)
        result = llmclient.generate( model=model_name, prompt=prompt_full, options=model_options, system=prompt_role)
        #pprint(result, compact=True, sort_dicts=False)
        bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result["response"])

        #JSON
        prompt_full = prompt_task_json + "".join(result["response"])
        result = llmclient.generate( model=model_name, prompt=prompt_full, options=model_options)
        #pprint(result, compact=True, sort_dicts=False)
        bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s")

        bid_utils.print_json_info_cctp(result["response"])


def run_list_model_chat(text_from_file):
    for i in range(len(list_model)):
        model_name = list_model[i]
        print("\n==========================================")
        print(f"===== Test model :{i+1}/{len(list_model)}: {model_name} =========\n")
        
        #Init system:
        result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])

        #Resumer
        prompt_full = prompt_task_resumer + "".join(text_from_file)
        #result = llmclient.generate(model=model_name, prompt=prompt_full, options=model_options, system=prompt_role)
        result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content': prompt_full}])
  
        pprint(result, compact=True, sort_dicts=False)
        #bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result["response"])
        bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result.message.content)

        #JSON
        #prompt_full = prompt_task_json + "".join(result["response"])
        #prompt_full = prompt_task_json + "".join(result.message.content)
        prompt_full = prompt_task_json + "".join(result.message.content)
        #result = llmclient.generate( model=model_name, prompt=prompt_full, options=model_options)
        result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full}])
                                                                                    
        #pprint(result, compact=True, sort_dicts=False)
        bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s")

        bid_utils.print_json_info_cctp(result.message.content)




In [19]:
## Liste all files for given consultation
## Consolide les fichiers bruts ou les resumés
## Run le modele pour resumer et json

# Preferable de faire une extraction par type de fichier (CCTP, CCAP, ..) avec un prompt dédié.
# donc regroupement par type de fichier

def run_1model_nCCTP_proj(model_name="gemma2:9b", nb_of_projet_to_test = 1):
    #model_name = "gemma2:9b"
    #nb_of_projet_to_test = 5
    count_id = 1

    # liste des ID correspondants au mask
    condition_mask = (df_consult_ebp['AO_docs'] == True) & \
                        (df_consult_ebp['AO_doc_type'] == "CCTP") 

    matching_ebp_ids = df_consult_ebp.loc[condition_mask, 'ID EBP'].unique()
    print(f"il y a {len(matching_ebp_ids)} projets avec au moins 1 document du type CCTP")
    print(f"on teste l'extraction avec {nb_of_projet_to_test} projets, par le modele {model_name}\n")

    #Init system:
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])

    if len(matching_ebp_ids) > 0:
        # Determine how many unique IDs we can actually select (min of 5 or available count)
        num_to_select = min(nb_of_projet_to_test, len(matching_ebp_ids))

        # Randomly select the desired number of unique EBP IDs
        selected_ebp_ids = np.random.choice(matching_ebp_ids, num_to_select, replace=False)

        #print(f"\n{num_to_select} ID(s) EBP aléatoire(s) sélectionné(s) correspondant au filtre:")
        for ebp_id in selected_ebp_ids:

            filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == ebp_id].iloc[0]
            print(f"\n{count_id}/{nb_of_projet_to_test} => ID:{ebp_id} statut_consult: {filtered_row['statut_consult']}")

            text_multi_files = ""
            list_file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == ebp_id) & (df_consult_ebp['AO_doc_type'] == 'CCTP')]['file_path']
            print(f"Il y a {len(list_file_consult)} fichiers dans la dossier AO consultation pour ce projet {ebp_id}")
            print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")
            for file_path in list_file_consult:
                nb_segment = file_path.split("\\")
                print("==> ", nb_segment[-1].strip())
                text_file = bid_utils.loadpdf_as_text(file_path.strip())
                text_multi_files += text_file + "\n" # Contatenate text
                
            if text_multi_files != "":
                print("Run model pour les fichier(s) selectionné(s)")

                #Resumer
                #prompt_full = prompt_task_resumer + "".join(text_multi_files)
                prompt_full = prompt_task_resumer + text_multi_files
                #result = llmclient.generate( model=model_name, prompt=prompt_full, options=model_options)
                result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full}])
                #pprint(result, compact=True, sort_dicts=False)
                bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n")

                #JSON
                #prompt_full = prompt_task_json + "".join(result["response"])
                #result = llmclient.generate( model=model_name, prompt=prompt_full, options=model_options)
                #prompt_full = prompt_task_json + "".join(result.message.content)
                prompt_full = prompt_task_json + result.message.content
                result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full}])
                #pprint(result, compact=True, sort_dicts=False)
                bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n")

                cctp_json = bid_utils.print_json_info_cctp(result.message.content)
                
                # extrait les infos du JSON
                if cctp_json != "":
                    parsed_json = {} # Initialize an empty dictionary
                    try:
                        parsed_json = json.loads(cctp_json)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON string: {e}")
                    except TypeError:
                        print("Input is not a string.") 
                    
                    # Extrait chaque clef
                    lieu_value = parsed_json.get("Lieu du Chantier")
                    type_travaux_value = parsed_json.get("Type de Travaux")
                    planning_value = parsed_json.get("Planning previsionnel")
                    duree_travaux_value = parsed_json.get("Durée Prévisionnelle des Travaux (en mois)")
                    prix_travaux_value = parsed_json.get("Prix des travaux (en euros)")
                    moa_value = parsed_json.get("Maitre ouvrage")
                    moe_value = parsed_json.get("Maitre oeuvre")

                    # Mise à jour du df_consult_elevated
                    mask = df_consult_elevated['ID EBP'] == ebp_id
                    if lieu_value is not None:
                        df_consult_elevated.loc[mask, 'lieu'] = lieu_value
                    if type_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'type travaux'] = type_travaux_value
                    if duree_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'duree travaux'] = duree_travaux_value
                    if planning_value is not None:
                        df_consult_elevated.loc[mask, 'planning'] = planning_value
                    if prix_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'prix travaux'] = prix_travaux_value
                    if moa_value is not None:
                        df_consult_elevated.loc[mask, 'maitre ouvrage'] = moa_value
                    if moe_value is not None:
                        df_consult_elevated.loc[mask, 'maitre oeuvre'] = moe_value
            else:
                print("Pas de fichiers consult à processer")
            count_id += 1
    else:
        print("\nAucune ligne trouvée avec 'statut_consult' == 'Chantiers'")



In [27]:
## Teste les modeles 

#list_model = ["gemma3:12b", "gemma2:9b", "llama3.1:8b", "mistral:7b"]
# gemma3:12b : very long , not performing so well.
# mistral:7b : does not respect instructions (info request, JSON structure)
list_model = ["llama3.1:8b", "gemma2:9b"]

# Select 1 file for the test
condition_mask = (df_consult_ebp['AO_docs'] == True) & \
                    (df_consult_ebp['AO_doc_type'] == "CCTP") 

matching_ebp_ids = df_consult_ebp.loc[condition_mask, 'ID EBP'].unique()
print(f"il y a {len(matching_ebp_ids)} projets avec au moins 1 document du type CCTP")
if len(matching_ebp_ids) > 0:
    # Determine how many unique IDs we can actually select (min of 5 or available count)
    #num_to_select = min(1, len(matching_ebp_ids))
    selected_ebp_id = np.random.choice(matching_ebp_ids, 1, replace=False)
    filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == selected_ebp_id[0]].iloc[0]
    print(f"ID:{selected_ebp_id} statut_consult: {filtered_row['statut_consult']}")
    print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")
    
    text_multi_files = ""
    file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == selected_ebp_id[0]) & (df_consult_ebp['AO_doc_type'] == 'CCTP')]['file_path'].iloc[0]
    print("File ==> ", file_consult)
    text_to_analyse = bid_utils.loadpdf_as_text(file_consult.strip())

# Compare model results for same file same prompt , models to test is defined in list_model
run_list_model_chat(text_to_analyse)   #best model so far: llama3.1:8b


il y a 350 projets avec au moins 1 document du type CCTP
ID:['no EBP 3731'] statut_consult: Devis
Ville:BREM SUR MER  Entreprise:ELAUCAMACLE 5  Mission:LE CHAMP PRIEUR
File ==>  C:\Users\jch_m\ATAE\Fabien ROUILLE - FABIEN\00-DEVIS\ARCHIVE DEVIS\DEVIS 2023\BREM SUR MER - ELAUCAMACLE 5 - LE CHAMP PRIEUR\Consultation\S4978 PA - LE CHAMP PRIEUR v3 2023.04.17-DCE EA.pdf


===== Test model :1/2: llama3.1:8b =========

ChatResponse(model='llama3.1:8b', created_at='2025-06-09T20:31:15.5078834Z', done=True, done_reason='stop', total_duration=61811867300, load_duration=12926100, prompt_eval_count=1307, prompt_eval_duration=36702490500, eval_count=269, eval_duration=25095418700, message=Message(role='assistant', content='Voici les éléments extraits du texte en respectant la structure demandée :\n\n**Nom Chantier**\n* Projet de la SAS ELAUCAMACLE 5\n* Lotissement "LE CHAMP PRIEUR"\n\n**Lieu du Chantier**\n* Ville : BREM-SUR-MER\n* Commune : BREM-SUR-MER\n* Département : inconnu (mais mentionné com

In [20]:
run_1model_nCCTP_proj(model_name="llama3.1:8b", nb_of_projet_to_test = 10)

il y a 350 projets avec au moins 1 document du type CCTP
on teste l'extraction avec 10 projets, par le modele llama3.1:8b


1/10 => ID:no EBP 14217 statut_consult: Devis
Il y a 2 fichiers dans la dossier AO consultation pour ce projet no EBP 14217
Ville:TOURS  Entreprise:USID  Mission:HOTEL DU GD Commandement
==>  CCTP_SPIE_SIGNE_AUBANEL.pdf
==>  CCTP_TPPL_SIGNE_AUBANEL.pdf
Run model pour les fichier(s) selectionné(s)

LLM response pour resumer: in 151s


LLM response pour JSON: in 77s


Accès aux données extraites :
Nom Chantier: 
Lieu du Chantier: Base de Défense de Tours
Maitre ouvrage: 
Maitre oeuvre: 
Type de Travaux: Modernisation, adaptation et rénovation de bâtiments
Planning previsionnel: 
Durée Prévisionnelle des Travaux (en mois): 
Prix des travaux (en euros): 
Categorie operation SPS: Information non disponible

2/10 => ID:210395 statut_consult: Archive
Il y a 2 fichiers dans la dossier AO consultation pour ce projet 210395
Ville:RSY  Entreprise:VENDEE EAU  Mission:Réservoi

In [22]:
len(df_consult_elevated[df_consult_elevated['type travaux'] != ''])
df_consult_elevated[df_consult_elevated['type travaux'] != ''].head(10)

,SPS Name,Ville,Entreprise,Mission,statut_consult,ID EBP,lieu,type travaux,planning,duree travaux,prix travaux,maitre ouvrage,maitre oeuvre
2988,Eric Garnier - ERIC,CHALLANS,SCCV BATICANA (LOTIPROMO),Clémenceau 4 MI,Devis,no EBP 610,,Construction,,0,0,,
5233,Eric Garnier - ERIC,BARBATRE,MAIRIE,Cimetière paysager La Martinière,Archive,220468,,"Voirie, réseaux et aménagements",,,,,
23708,Fabien ROUILLE - FABIEN,RSY,PC DE VENDEE,Extension bâtiment,Archive,220127,,Réhabilitation d'un bâtiment,,,,,
23838,Fabien ROUILLE - FABIEN,RSY,VENDEE EAU,Réservoirs sur tour ZI NORD,Archive,210395,,Construction d'une station d'eau,,,,,
86767,Nicolas PATRY - NICOLAS PATRY,TOURS,CHRU TOURS,Nouvel Hôpital Psychiatrique,Perdu,no EBP 14171,,Prestations intellectuelles,,,,,
86998,Nicolas PATRY - NICOLAS PATRY,TOURS,USID,HOTEL DU GD Commandement,Devis,no EBP 14217,Base de Défense de Tours,"Modernisation, adaptation et rénovation de bât...",,,,,
107225,Nicolas POTIER - Nicolas POTIER,LORIENT,MH,Construction immeuble 50 Logts Maison Santé St...,Chantier,250101,,Construction,,0,0,,
142468,Stéphanie LAFORGE - STEPHANIE,GUERANDE,CAP ATLANTIQUE,KERHILIERS,Perdu,no EBP 22116,Parc d'activités de Villejames,Requalification de voirie,,,,,
